## 01 – Data Cleaning dos focos de queimadas em MA (2025)

**Objetivo:**  
Limpar, padronizar e validar os dados brutos de focos de queimadas em Maranhão (jan–jun/2025), preparando o CSV para as próximas etapas de análise e EDA.

### Importar Bibliotecas e Configurações (Code)

In [1]:
import pandas as pd
import numpy as np
import os
os.getcwd()

# Ajustes de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

### Carregar o CSV

In [2]:
df = pd.read_csv('../data/raw/focos_mensal_ma_2025.csv')

###  Inspeção Inicial

In [3]:
# Shape e colunas
print("Linhas x Colunas:", df.shape)
display(df.dtypes)

Linhas x Colunas: (34791, 16)


id                        object
lat                      float64
lon                      float64
data_hora_gmt             object
satelite                  object
municipio                 object
estado                    object
pais                      object
municipio_id               int64
estado_id                  int64
pais_id                    int64
numero_dias_sem_chuva    float64
precipitacao             float64
risco_fogo               float64
bioma                     object
frp                      float64
dtype: object

In [4]:

# Primeiros registros
display(df.head())

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
0,c406b4a8-0f9c-3c1e-9a83-f9d032adefee,-5.21240,-44.90449,2025-01-01 3:46:00,NOAA-20,BARRA DO CORDA,MARANHÃO,Brasil,2101608,21,33,0.0,0.35,0.24,Cerrado,4.3
1,261abf1d-1a28-3e86-bf99-032ee0ca4419,-1.68696,-45.87955,2025-01-01 4:38:00,NOAA-21,AMAPÁ DO MARANHÃO,MARANHÃO,Brasil,2100550,21,33,1.0,0.10,0.33,Amazônia,0.7
2,517e242c-ff97-3932-bdd4-13544c77d3e1,-2.18130,-45.67765,2025-01-01 4:38:00,NOAA-21,GOVERNADOR NUNES FREIRE,MARANHÃO,Brasil,2104677,21,33,2.0,0.00,0.91,Amazônia,2.1
3,11f9154d-60cb-34fe-8256-b3a4dba8b7b2,-1.68747,-45.87200,2025-01-01 4:38:00,NOAA-21,AMAPÁ DO MARANHÃO,MARANHÃO,Brasil,2100550,21,33,1.0,0.10,0.72,Amazônia,0.9
4,c92df387-c2d1-345b-9290-47b2300985e0,-1.68975,-45.88038,2025-01-01 4:38:00,NOAA-21,AMAPÁ DO MARANHÃO,MARANHÃO,Brasil,2100550,21,33,1.0,0.10,0.34,Amazônia,1.2


In [5]:

# Missing & duplicatas
print("NAs por coluna:\n", df.isna().sum())
print()
print("----------------------------")
print()
print("Duplicatas totais:", df.duplicated().sum())

NAs por coluna:
 id                         0
lat                        0
lon                        0
data_hora_gmt              0
satelite                   0
municipio                  0
estado                     0
pais                       0
municipio_id               0
estado_id                  0
pais_id                    0
numero_dias_sem_chuva    542
precipitacao             542
risco_fogo               542
bioma                      0
frp                      996
dtype: int64

----------------------------

Duplicatas totais: 0


### Tratamento de Colunas de Data
- Converter a string em `datetime`, criando colunas auxiliares de ano e mês.

In [6]:
df['data_hora_gmt'] = pd.to_datetime(df['data_hora_gmt'], errors='coerce')
df['ano']   = df['data_hora_gmt'].dt.year
df['mes']   = df['data_hora_gmt'].dt.month
df['dia']   = df['data_hora_gmt'].dt.day
df['hora']  = df['data_hora_gmt'].dt.hour

# Validar
display(df[['data_hora_gmt','ano','mes', 'dia', 'hora']].head())

,data_hora_gmt,ano,mes,dia,hora
0,2025-01-01 03:46:00,2025,1,1,3
1,2025-01-01 04:38:00,2025,1,1,4
2,2025-01-01 04:38:00,2025,1,1,4
3,2025-01-01 04:38:00,2025,1,1,4
4,2025-01-01 04:38:00,2025,1,1,4


### Remoção de Duplicatas
Definimos duplicatas por combinação (`id`) ou por (`lat`, `lon`, `data_hora_gmt`).

In [7]:
# Exemplo: por id
df = df.drop_duplicates(subset=['id'])

# ou, se preferir, pela combinação geotemporal
# df = df.drop_duplicates(subset=['lat','lon','data_hora_gmt'])

print("Duplicatas após remoção:", df.duplicated().sum())

Duplicatas após remoção: 0


### Padronização de Strings

In [8]:
df['estado']    = df['estado'].str.strip().str.upper()
df['municipio'] = df['municipio'].str.strip().str.title()
df['satelite']  = df['satelite'].str.strip().str.upper()

display(df[['estado','municipio','satelite']].drop_duplicates().head(10))

,estado,municipio,satelite
0,MARANHÃO,Barra Do Corda,NOAA-20
1,MARANHÃO,Amapá Do Maranhão,NOAA-21
2,MARANHÃO,Governador Nunes Freire,NOAA-21
6,MARANHÃO,Presidente Médici,NOAA-21
8,MARANHÃO,Mata Roma,NOAA-21
9,MARANHÃO,Godofredo Viana,NOAA-21
13,MARANHÃO,Alto Alegre Do Pindaré,NOAA-21
14,MARANHÃO,Barra Do Corda,NOAA-21
16,MARANHÃO,Grajaú,NOAA-21
17,MARANHÃO,Magalhães De Almeida,TERRA_M-T


### Tratamento de Valores Ausentes

In [9]:
# Inspecionar proporção de NAs
na_pct = df.isna().mean() * 100
print(na_pct[na_pct>0])

# Exemplo de preenchimento
df['numero_dias_sem_chuva'] = df['numero_dias_sem_chuva'].fillna(0)
df['precipitacao']          = df['precipitacao'].fillna(0)
df['risco_fogo']            = df['risco_fogo'].fillna(df['risco_fogo'].median())

# FRP tem NAs: podemos preencher com 0 ou remover
df['frp'] = df['frp'].fillna(0)

numero_dias_sem_chuva    1.557874
precipitacao             1.557874
risco_fogo               1.557874
frp                      2.862809
dtype: float64


### Checagem Geográfica
- Assegurar que `lat` e `lon` estejam dentro dos limites de Maranhão.

In [10]:
lat_min, lat_max = -10.5, -1.0
lon_min, lon_max = -49.0, -40.5

mask_geo = (
    df['lat'].between(lat_min, lat_max) &
    df['lon'].between(lon_min, lon_max)
)
print("Fora do MA:", (~mask_geo).sum())
df = df[mask_geo]

Fora do MA: 0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34791 entries, 0 to 34790
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     34791 non-null  object        
 1   lat                    34791 non-null  float64       
 2   lon                    34791 non-null  float64       
 3   data_hora_gmt          34791 non-null  datetime64[ns]
 4   satelite               34791 non-null  object        
 5   municipio              34791 non-null  object        
 6   estado                 34791 non-null  object        
 7   pais                   34791 non-null  object        
 8   municipio_id           34791 non-null  int64         
 9   estado_id              34791 non-null  int64         
 10  pais_id                34791 non-null  int64         
 11  numero_dias_sem_chuva  34791 non-null  float64       
 12  precipitacao           34791 non-null  float64       
 13  r

### Resumo Final e Exportação
Salvaremos o CSV em `data/processed/` com versão ou timestamp.

In [13]:
from datetime import datetime
versao = datetime.now().strftime('%Y%m%d_%H%M')
path = f"../data/processed/queimadas_MA_2025_clean_v{versao}.csv"
df.to_csv(path, index=False)
print("Arquivo salvo em:", path)

Arquivo salvo em: ../data/processed/queimadas_MA_2025_clean_v20250628_2146.csv
